In [1]:
import pandas as pd
import altair as alt
import numpy as np
import sklearn
from sklearn.decomposition import PCA

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Creating data frames
fedRate = pd.read_csv('/content/drive/MyDrive/Group 3 Project/Datasets/FEDFUNDSBYYEAR.csv')

mortRate = pd.read_csv("/content/drive/MyDrive/Group 3 Project/Datasets/30fymByYear.csv")

autoRate = pd.read_csv("/content/drive/MyDrive/Group 3 Project/Datasets/AutoFinanceInterestRate.csv")

creditRate = pd.read_csv("/content/drive/MyDrive/Group 3 Project/Datasets/CreditCardInterest.csv")

inflationRate = pd.read_csv("/content/drive/MyDrive/Group 3 Project/Datasets/InflationRate.csv")

mortDelinquencyRate = pd.read_csv("/content/drive/MyDrive/Group 3 Project/Datasets/MortDelinquency.csv")


In [4]:
mortRate

,observation_date,MORTGAGE30US
0,2000-01-01,8.05
1,2001-01-01,6.97
2,2002-01-01,6.54
3,2003-01-01,5.83
4,2004-01-01,5.84
5,2005-01-01,5.87
6,2006-01-01,6.41
7,2007-01-01,6.34
8,2008-01-01,6.03
9,2009-01-01,5.04


In [5]:
# THESE WILL HAVE TO EDITED FOR CHART CLARITY LATER



# global x-axis for time serieS
all_dates = pd.concat([
    fedRate['observation_date'],
    mortRate["observation_date"],
    creditRate["observation_date"],
    autoRate['observation_date'],
    inflationRate["observation_date"],
    mortDelinquencyRate["observation_date"]
])

x_scale = alt.Scale(domain = [all_dates.min(), all_dates.max()])



# create global y-axis for all rates
all_rates = pd.concat([
    fedRate["FEDFUNDS"],
    mortRate['MORTGAGE30US'],
    creditRate["TERMCBCCALLNS"],
    autoRate["TERMCBAUTO48NS"],
    inflationRate["FPCPITOTLZGUSA"],
    mortDelinquencyRate["DRSFRMACBS"]
])

y_scale = alt.Scale(domain = [all_rates.min(), all_rates.max()])

In [6]:
import pandas as pd

# Read CSVs
fedRate = pd.read_csv('/content/drive/MyDrive/Group 3 Project/Datasets/FEDFUNDSBYYEAR.csv')
mortRate = pd.read_csv("/content/drive/MyDrive/Group 3 Project/Datasets/30fymByYear.csv")
autoRate = pd.read_csv("/content/drive/MyDrive/Group 3 Project/Datasets/AutoFinanceInterestRate.csv")
creditRate = pd.read_csv("/content/drive/MyDrive/Group 3 Project/Datasets/CreditCardInterest.csv")
inflationRate = pd.read_csv("/content/drive/MyDrive/Group 3 Project/Datasets/InflationRate.csv")
mortDelinquencyRate = pd.read_csv("/content/drive/MyDrive/Group 3 Project/Datasets/MortDelinquency.csv")

# Standardize column names
def prepare_rate_df(df, value_col, category_name):
    return df[['observation_date', value_col]].rename(columns={value_col: 'rate'}).assign(category=category_name)

# Prepare each individual rate DataFrame
mort_df = prepare_rate_df(mortRate, 'MORTGAGE30US', 'Mortgage Rate')
auto_df = prepare_rate_df(autoRate, 'TERMCBAUTO48NS', 'Auto Loan Rate')
credit_df = prepare_rate_df(creditRate, 'TERMCBCCALLNS', 'Credit Card Rate')
fed_df = prepare_rate_df(fedRate, 'FEDFUNDS', 'Federal Funds Rate')
inflation_df = prepare_rate_df(inflationRate, 'FPCPITOTLZGUSA', 'Inflation Rate')
delinq_df = prepare_rate_df(mortDelinquencyRate, 'DRSFRMACBS', 'Mortgage Delinquency Rate')

# Combine all into one long-form DataFrame
combined_df = pd.concat([mort_df, auto_df, credit_df, fed_df, inflation_df, delinq_df], ignore_index=True)

In [7]:
# POTENTIALLY CREATE ONE DATAFRAME SO THAT i CAN EASILY EDIT THE CHARTS FOR THIS

# Create brush selection for the x-axis (time)
brush_selection = alt.selection_interval(encodings=['x'], empty='none')

# FED Rate Line
baseFedLine = alt.Chart(fedRate).mark_line(strokeWidth=2, color='lightgray').encode(
    x=alt.X("observation_date:T", scale=x_scale),
    y="FEDFUNDS:Q"
)

highlightFedLine = alt.Chart(fedRate).mark_line(strokeWidth=5, color='red').encode(
    x=alt.X("observation_date:T", scale=x_scale),
    y="FEDFUNDS:Q"
).transform_filter(
    brush_selection
)

fedLine = (baseFedLine + highlightFedLine).add_params(brush_selection)

# Inflation Line
baseInflationLine = alt.Chart(inflationRate).mark_line(strokeWidth=2, color='lightgray').encode(
    x=alt.X("observation_date:T", scale=x_scale),
    y="FPCPITOTLZGUSA"
)

highlightInflationLine = alt.Chart(inflationRate).mark_line(strokeWidth=7, color='#f54295').encode(
    x=alt.X("observation_date:T", scale=x_scale),
    y="FPCPITOTLZGUSA"
).transform_filter(
    brush_selection
)

inflationLine = baseInflationLine + highlightInflationLine

# Mortgage Rate Line
baseMortLine = alt.Chart(mortRate).mark_line(color="lightgray").encode(
    x=alt.X("observation_date:T", scale=x_scale),
    y="MORTGAGE30US"
)

highlightMortLine = alt.Chart(mortRate).mark_line(color="blue").encode(
    x=alt.X("observation_date:T", scale=x_scale),
    y="MORTGAGE30US"
).transform_filter(
    brush_selection
)

mortLine = baseMortLine + highlightMortLine

# Auto Rate Line
baseAutoLine = alt.Chart(autoRate).mark_line(color="lightgray").encode(
    x=alt.X("observation_date:T", scale=x_scale),
    y="TERMCBAUTO48NS"
)

highlightAutoLine = alt.Chart(autoRate).mark_line(color="green").encode(
    x=alt.X("observation_date:T", scale=x_scale),
    y="TERMCBAUTO48NS"
).transform_filter(
    brush_selection
)

autoLine = baseAutoLine + highlightAutoLine

# Credit Rate Line
baseCreditLine = alt.Chart(creditRate).mark_line(color="lightgray").encode(
    x=alt.X("observation_date:T", scale=x_scale),
    y="TERMCBCCALLNS"
)

highlightCreditLine = alt.Chart(creditRate).mark_line(color="purple").encode(
    x=alt.X("observation_date:T", scale=x_scale),
    y="TERMCBCCALLNS"
).transform_filter(
    brush_selection
)

creditLine = baseCreditLine + highlightCreditLine

# Merge interest product charts
intProdchart = (mortLine + autoLine + creditLine).properties(
    width=650,
    height=650
)

# Merge fed and inflation rates chart
fedRates = (fedLine + inflationLine).properties(
    width=650,
    height=650
)

# Display side by side
fedRates | intProdchart


alt.HConcatChart(...)

FINAL RATES COMPARISON

In [8]:
import altair as alt

all_dates = combined_df['observation_date']
x_scale = alt.Scale(domain=[all_dates.min(), all_dates.max()])

brush_selection = alt.selection_interval(encodings=['x'], empty='all')
highlight = alt.selection_point(fields=["category"], bind="legend")

base_chart = alt.Chart(combined_df).mark_line().encode(
    x=alt.X("observation_date:T", scale=x_scale),
    y=alt.Y("rate:Q"),
    color=alt.condition(
        highlight,
        alt.Color("category:N", legend=alt.Legend(title="Rate Type")),
        alt.value("lightgray")
    ),
    tooltip=["observation_date:T", "rate:Q", "category:N"],
    opacity=alt.condition(highlight, alt.value(1), alt.value(0.4)),
    strokeWidth=alt.condition(highlight, alt.value(3), alt.value(1.2))
).add_params(
    brush_selection,
    highlight
).transform_filter(
    brush_selection
)

fed_rates = base_chart.transform_filter(
    alt.FieldOneOfPredicate(field="category", oneOf=[
        "Federal Funds Rate", "Inflation Rate"
    ])
).properties(width=500 , height=300)

int_prod_chart = base_chart.transform_filter(
    alt.FieldOneOfPredicate(field="category", oneOf=[
        "Mortgage Rate", "Auto Loan Rate", "Credit Card Rate", "Mortgage Delinquency Rate"
    ])
).properties(width=500, height=300)

fed_and_interest = (fed_rates & int_prod_chart).properties(
    bounds="flush",
    padding={'right': 50}
).configure(
    autosize=alt.AutoSizeParams(
        type="pad",
        contains="padding"
    )
)
fed_and_interest

alt.VConcatChart(...)

In [9]:
all_dates = combined_df['observation_date']
x_scale = alt.Scale(domain=[all_dates.min(), all_dates.max()])

brush_selection = alt.selection_interval(encodings=['x'], empty='all')
highlight = alt.selection_point(fields=["category"], bind="legend")

base_chart = alt.Chart(combined_df).mark_line().encode(
    x=alt.X("observation_date:T", scale=x_scale, title=None),
    y=alt.Y("rate:Q", title="Rate (%)"),
    color=alt.condition(
        highlight,
        alt.Color("category:N", legend=alt.Legend(title="Rate Type")),
        alt.value("lightgray")
    ),
    tooltip=["observation_date:T", "rate:Q", "category:N"],
    opacity=alt.condition(highlight, alt.value(1), alt.value(0.4)),
    strokeWidth=alt.condition(highlight, alt.value(3), alt.value(1.2))
).add_params(
    brush_selection,
    highlight
).transform_filter(
    brush_selection
)

fed_rates = base_chart.transform_filter(
    alt.FieldOneOfPredicate(field="category", oneOf=[
        "Federal Funds Rate", "Inflation Rate"
    ])
).properties(
    width=500,
    height=300
)

int_prod_chart = base_chart.transform_filter(
    alt.FieldOneOfPredicate(field="category", oneOf=[
        "Mortgage Rate", "Auto Loan Rate", "Credit Card Rate", "Mortgage Delinquency Rate"
    ])
).properties(
    width=500,
    height=300
)

fed_and_interest = (fed_rates & int_prod_chart).properties(
    bounds="flush",
    padding={'right': 50}
).configure(
    autosize=alt.AutoSizeParams(
        type="pad",
        contains="padding"
    )
)

fed_and_interest

alt.VConcatChart(...)

In [10]:
import altair as alt

all_dates = combined_df['observation_date']
x_scale = alt.Scale(domain=[all_dates.min(), all_dates.max()])

brush_selection = alt.selection_interval(encodings=['x'], empty='all')
highlight = alt.selection_point(fields=["category"], bind="legend")

base_chart = alt.Chart(combined_df).mark_line().encode(
    x=alt.X("observation_date:T", scale=x_scale, title=None),
    y=alt.Y("rate:Q", title="Rate (%)"),
    color=alt.condition(
        highlight,
        alt.Color(
            "category:N",
            scale=alt.Scale(
                domain=[
                    "Federal Funds Rate",
                    "Inflation Rate",
                    "Mortgage Rate",
                    "Auto Loan Rate",
                    "Credit Card Rate",
                ],
                range=[
                    "#007A7A",  # Federal Funds Rate — blue/cyan
                    "#CB0000",  # Inflation Rate — red
                    "#FF7400",  # Mortgage Rate — orange
                    "#00DADA",  # Auto Loan Rate — light blue
                    "#00A300",  # Credit Card Rate — neon green

                ]
            ),
            legend=alt.Legend(title="Rate Type")
        ),
        alt.value("lightgray")
    ),
    tooltip=["observation_date:T", "rate:Q", "category:N"],
    opacity=alt.condition(highlight, alt.value(1), alt.value(0.4)),
    strokeWidth=alt.condition(highlight, alt.value(3), alt.value(1.2))
).add_params(
    brush_selection,
    highlight
).transform_filter(
    brush_selection
)

fed_rates = base_chart.transform_filter(
    alt.FieldOneOfPredicate(field="category", oneOf=[
        "Federal Funds Rate", "Inflation Rate"
    ])
).properties(
    width=500,
    height=300
)

int_prod_chart = base_chart.transform_filter(
    alt.FieldOneOfPredicate(field="category", oneOf=[
        "Mortgage Rate", "Auto Loan Rate", "Credit Card Rate"
    ])
).properties(
    width=500,
    height=300
)

fed_and_interest = (fed_rates & int_prod_chart).properties(
    bounds="flush",
    padding={'right': 50}
).configure(
    autosize=alt.AutoSizeParams(
        type="pad",
        contains="padding"
    )
)

fed_and_interest


alt.VConcatChart(...)

In [ ]:
html = fed_and_interest.to_html()

with open('/content/fed_and_interest.html', 'w') as f:
    f.write(html)

In [ ]:
from google.colab import files
files.download('/content/fed_and_interest.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [11]:


# Fed rate takes around 12 months to affect inflation rate.
fedAinflation = inflationRate.copy()
fedAinflation["FPCPITOTLZGUSA"] = fedAinflation["FPCPITOTLZGUSA"].shift(-1)

# Get the fed rate data and add it to new dataframe

fedAinflation['FedRate'] = fedRate['FEDFUNDS']

fedAinflation = fedAinflation.dropna()

fedAinflation = fedAinflation.rename(columns={"FPCPITOTLZGUSA": "InflationRate"})

fedAinflation["InflationRate"] = fedAinflation["InflationRate"].round(2)

fedAinflation

,observation_date,InflationRate,FedRate
0,2000-01-01,2.83,6.24
1,2001-01-01,1.59,3.89
2,2002-01-01,2.27,1.67
3,2003-01-01,2.68,1.13
4,2004-01-01,3.39,1.35
5,2005-01-01,3.23,3.21
6,2006-01-01,2.85,4.96
7,2007-01-01,3.84,5.02
8,2008-01-01,-0.36,1.93
9,2009-01-01,1.64,0.16


In [12]:
# heat map of inflation and fed rate



heatmap = alt.Chart(fedAinflation).mark_rect().encode(
    alt.X('InflationRate:Q', bin=alt.Bin(maxbins=30)),
    alt.Y('FedRate:Q', bin=alt.Bin(maxbins=30)),
    alt.Color('count():Q', scale=alt.Scale(scheme='viridis')),
    tooltip=['count()']
).properties(
    title='Heatmap: Inflation Rate vs Fed Rate'
)

heatmap


alt.Chart(...)

In [13]:
correlation = fedAinflation['FedRate'].corr(fedAinflation['InflationRate'])
correlation

np.float64(0.04371923078195211)

In [14]:


# Create brush
new_brush = alt.selection_interval(encodings = ['x'])

# create correlation scatter with inflation and fed rates
scatter = alt.Chart(fedAinflation).mark_circle(size = 80).encode(
    x = "FedRate",
    y= "InflationRate",
    tooltip = ["InflationRate", "FedRate",'observation_date:T'],
).properties(
    width=600,
    height = 600
).transform_filter(
    new_brush
)

# create regression line
regression = scatter.transform_regression(
    'FedRate', 'InflationRate'
).mark_line(color = 'red')


# regression + correlation scatter
final = (scatter + regression).properties(
    width = 600,
    height = 600,
)

# create federal funds rate line
fed_line = alt.Chart(fedAinflation).mark_line(color = 'red').encode(
    x = "observation_date:T",
    y = "FedRate:Q",
    tooltip = ['observation_date:T','FedRate:Q']
)

# create adjusted inflation rate line
inflation_line = alt.Chart(fedAinflation).mark_line(color = 'green').encode(
    x = 'observation_date:T',
    y = "InflationRate:Q",
    tooltip = ['observation_date:T','InflationRate:Q'],
)

# inflation line + fed line over time final chart
timeSeries = (inflation_line + fed_line).properties(
    width = 500,
    height = 300
).add_params(
    new_brush
)
timeSeries | final.properties(
    width = 500,
    height = 300
)

alt.HConcatChart(...)

In [15]:
fedAinflation

,observation_date,InflationRate,FedRate
0,2000-01-01,2.83,6.24
1,2001-01-01,1.59,3.89
2,2002-01-01,2.27,1.67
3,2003-01-01,2.68,1.13
4,2004-01-01,3.39,1.35
5,2005-01-01,3.23,3.21
6,2006-01-01,2.85,4.96
7,2007-01-01,3.84,5.02
8,2008-01-01,-0.36,1.93
9,2009-01-01,1.64,0.16


In [16]:
# "Real-ish inflation rate"
real_inflation_df = fedAinflation.copy()

real_inflation_df['RealRate'] = real_inflation_df["FedRate"]- real_inflation_df["InflationRate"]

real_inflation_df

real_rate = alt.Chart(real_inflation_df).mark_line(color = "purple").encode(
    x = "observation_date:T",
    y = "RealRate:Q"
)
inflat = alt.Chart(real_inflation_df).mark_line(color = 'red').encode(
    x = "observation_date:T",
    y = "InflationRate:Q"
)

real_fed = alt.Chart(real_inflation_df).mark_line(color = "blue").encode(
    x = "observation_date:T",
    y = "FedRate:Q"
)

(real_rate + inflat + real_fed).properties(
    width = 500,
    height = 300
)


alt.LayerChart(...)

In [17]:
# Gap from 2 percent target of inflation

# create new dataframe that has target gaps
policy_df = fedAinflation.copy()

# Trackes whether inflation itself is at target or below target
policy_df["InflationGap"] = policy_df["InflationRate"] - 2

# If policyGap > 0 the fed is acting more stringently
# If policyGap < 0 the fed might be behind the curve
policy_df["PolicyGap"] = policy_df["FedRate"] - policy_df["InflationGap"]

# create one melted dataframe for the inflation and fed
melted_government = policy_df[["observation_date","InflationRate", 'FedRate']].copy()

melted_government = melted_government.melt(
    id_vars="observation_date",
    value_vars=["InflationRate", "FedRate"],
    var_name="Metric",
    value_name="Value"
)

# create another melt chart just for the target gap and the inflation gap
gap_df = policy_df[["observation_date", "InflationGap", "PolicyGap"]].copy()

gap_df = gap_df.melt(
    id_vars="observation_date",
    value_vars=["InflationGap", "PolicyGap"],
    var_name="GapMetric",
    value_name="Value"
)

gov_color_scale = alt.Scale(
    domain=["InflationRate", "FedRate"],
    range=["red", "royalblue"]
)

gap_color_scale = alt.Scale(
    domain=["InflationGap", "PolicyGap"],
    range=["purple", "orange"]
)

# for chart on the left
melt_gov = alt.Chart(melted_government).mark_line().encode(
    x="observation_date:T",
    y="Value:Q",
    color=alt.Color("Metric:N", scale=gov_color_scale,
        legend=alt.Legend(title="Metric"))
).properties(
    width=400,
    height=300,
    title="Inflation and the Federal Funds Rate"
)

# Add horizontal line at y=0 (2% inflation target on InflationGap scale)
target_line = alt.Chart(pd.DataFrame({"y": [0]})).mark_rule(
    color="gray",
    strokeDash=[4, 4]
).encode(
    y="y:Q"
)

# Label the target line
target_label = alt.Chart(pd.DataFrame({
    "x": [policy_df["observation_date"].max()],
    "y": [0],
    "text": ["Target (2% inlation)"]
})).mark_text(
    align="left",
    baseline="bottom",
    dx=5,
    dy=-5,
    color="black"
).encode(
    x="x:T",
    y="y:Q",
    text="text:N"
)

# for chart on the right
gap_lines = alt.Chart(gap_df).mark_line().encode(
    x="observation_date:T",
    y="Value:Q",
    color=alt.Color("GapMetric:N", scale=gap_color_scale,
        legend=alt.Legend(title="Gap Type"))
).properties(
    width=400,
    height=300,
    title=""
)

# combine gap lines with the target line and label
gap_chart = alt.layer(gap_lines, target_line, target_label)

true_label = alt.Chart(pd.DataFrame({
    "x": [policy_df["observation_date"].max()],
    "y": [2],
    "text": ["Target (2% inlation)"]
})).mark_text(
    align="left",
    baseline="bottom",
    dx=5,
    dy=-5,
    color="black"
).encode(
    x="x:T",
    y="y:Q",
    text="text:N"
)

melt_got_line = alt.layer(melt_gov, true_label)

# combine charts with fully independent color scales
(melt_got_line | gap_chart).resolve_scale(color='independent')


alt.HConcatChart(...)

In [18]:
# Gap from 2 percent target of inflation

# create new dataframe that has target gaps
policy_df = fedAinflation.copy()

# Trackes whether inflation itself is at target or below target
policy_df["InflationGap"] = policy_df["InflationRate"] - 2

# If policyGap > 0 the fed is acting more stringently
# If policyGap < 0 the fed might be behind the curve
policy_df["PolicyGap"] = policy_df["FedRate"] - policy_df["InflationGap"]

# create one melted dataframe for the inflation and fed
melted_government = policy_df[["observation_date","InflationRate", 'FedRate']].copy()

melted_government = melted_government.melt(
    id_vars="observation_date",
    value_vars=["InflationRate", "FedRate"],
    var_name="Metric",
    value_name="Value"
)

# create another melt chart just for the target gap and the inflation gap
gap_df = policy_df[["observation_date", "InflationGap", "PolicyGap"]].copy()

gap_df = gap_df.melt(
    id_vars="observation_date",
    value_vars=["InflationGap", "PolicyGap"],
    var_name="GapMetric",
    value_name="Value"
)

gov_color_scale = alt.Scale(
    domain=["InflationRate", "FedRate"],
    range=["red", "royalblue"]
)

gap_color_scale = alt.Scale(
    domain=["InflationGap", "PolicyGap"],
    range=["purple", "orange"]
)

# for chart on the left
melt_gov = alt.Chart(melted_government).mark_line().encode(
    x="observation_date:T",
    y="Value:Q",
    color=alt.Color("Metric:N", scale=gov_color_scale,
        legend=alt.Legend(title="Metric"))
).properties(
    width=400,
    height=300,
    title="Inflation and the Federal Funds Rate"
)

# Add solid black line at y = 2% on the left chart
solid_target_line = alt.Chart(pd.DataFrame({"y": [2]})).mark_rule(
    color="black"
).encode(
    y="y:Q"
)

# Add horizontal line at y=0 (2% inflation target on InflationGap scale)
target_line = alt.Chart(pd.DataFrame({"y": [0]})).mark_rule(
    color="gray",
    strokeDash=[4, 4]
).encode(
    y="y:Q"
)

# Label the target line
target_label = alt.Chart(pd.DataFrame({
    "x": [policy_df["observation_date"].max()],
    "y": [0],
    "text": ["Target (2% inlation)"]
})).mark_text(
    align="left",
    baseline="bottom",
    dx=5,
    dy=-5,
    color="black"
).encode(
    x="x:T",
    y="y:Q",
    text="text:N"
)

# for chart on the right
gap_lines = alt.Chart(gap_df).mark_line().encode(
    x="observation_date:T",
    y="Value:Q",
    color=alt.Color("GapMetric:N", scale=gap_color_scale,
        legend=alt.Legend(title="Gap Type"))
).properties(
    width=400,
    height=300,
    title=""
)

# combine gap lines with the target line and label
gap_chart = alt.layer(gap_lines, target_line, target_label)

true_label = alt.Chart(pd.DataFrame({
    "x": [policy_df["observation_date"].max()],
    "y": [2],
    "text": ["Target (2% inlation)"]
})).mark_text(
    align="left",
    baseline="bottom",
    dx=5,
    dy=-5,
    color="black"
).encode(
    x="x:T",
    y="y:Q",
    text="text:N"
)

melt_got_line = alt.layer(melt_gov, true_label, solid_target_line)

# combine charts with fully independent color scales
policy_and_inflation_target = (melt_got_line | gap_chart).resolve_scale(color='independent')

policy_and_inflation_target

alt.HConcatChart(...)

In [19]:
# Gap from 2 percent target of inflation

# create new dataframe that has target gaps
policy_df = fedAinflation.copy()

# Trackes whether inflation itself is at target or below target
policy_df["InflationGap"] = policy_df["InflationRate"] - 2

# If policyGap > 0 the fed is acting more stringently
# If policyGap < 0 the fed might be behind the curve
policy_df["PolicyGap"] = policy_df["FedRate"] - policy_df["InflationGap"]

# create one melted dataframe for the inflation and fed
melted_government = policy_df[["observation_date","InflationRate", 'FedRate']].copy()

melted_government = melted_government.melt(
    id_vars="observation_date",
    value_vars=["InflationRate", "FedRate"],
    var_name="Metric",
    value_name="Value"
)

# create another melt chart just for the target gap and the inflation gap
gap_df = policy_df[["observation_date", "InflationGap", "PolicyGap"]].copy()

gap_df = gap_df.melt(
    id_vars="observation_date",
    value_vars=["InflationGap", "PolicyGap"],
    var_name="GapMetric",
    value_name="Value"
)

gov_color_scale = alt.Scale(
    domain=["InflationRate", "FedRate"],
    range=["red", "royalblue"]
)

gap_color_scale = alt.Scale(
    domain=["InflationGap", "PolicyGap"],
    range=["purple", "orange"]
)

# for chart on the left
melt_gov = alt.Chart(melted_government).mark_line().encode(
    x="observation_date:T",
    y="Value:Q",
    color=alt.Color("Metric:N", scale=gov_color_scale,
        legend=alt.Legend(title="Metric"))
).properties(
    width=400,
    height=300,
    title="Inflation and the Federal Funds Rate"
)

# Add dashed black line at y = 2% on the left chart
solid_target_line = alt.Chart(pd.DataFrame({"y": [2]})).mark_rule(
    color="black",
    strokeDash=[4, 4]
).encode(
    y="y:Q"
)

# Add dashed gray line at y=0 (2% inflation target on InflationGap scale)
target_line = alt.Chart(pd.DataFrame({"y": [0]})).mark_rule(
    color="gray",
    strokeDash=[4, 4]
).encode(
    y="y:Q"
)

# Label the target line (right chart)
target_label = alt.Chart(pd.DataFrame({
    "x": [policy_df["observation_date"].max()],
    "y": [0],
    "text": ["Target (2% inflation)"]
})).mark_text(
    align="left",
    baseline="bottom",
    dx=5,
    dy=-5,
    color="black"
).encode(
    x="x:T",
    y="y:Q",
    text="text:N"
)

# for chart on the right
gap_lines = alt.Chart(gap_df).mark_line().encode(
    x="observation_date:T",
    y="Value:Q",
    color=alt.Color("GapMetric:N", scale=gap_color_scale,
        legend=alt.Legend(title="Gap Type"))
).properties(
    width=400,
    height=300,
    title=""
)

# combine gap lines with the target line and label
gap_chart = alt.layer(gap_lines, target_line, target_label)

# Label the target line (left chart)
true_label = alt.Chart(pd.DataFrame({
    "x": [policy_df["observation_date"].max()],
    "y": [2],
    "text": ["Target (2% inflation)"]
})).mark_text(
    align="left",
    baseline="bottom",
    dx=5,
    dy=-5,
    color="black"
).encode(
    x="x:T",
    y="y:Q",
    text="text:N"
)

# layer both the lines and label on the left chart
melt_got_line = alt.layer(melt_gov, solid_target_line, true_label)

# combine charts with fully independent color scales
(melt_got_line & gap_chart).resolve_scale(color='independent')


alt.VConcatChart(...)

In [20]:
# Gap from 2 percent target of inflation

nearest = alt.selection_point(
    nearest=True,
    on="pointerover",
    fields=["observation_date"],
    empty=False
)

# create new dataframe that has target gaps
policy_df = fedAinflation.copy()

# Trackes whether inflation itself is at target or below target
policy_df["InflationGap"] = policy_df["InflationRate"] - 2

# If policyGap > 0 the fed is acting more stringently
# If policyGap < 0 the fed might be behind the curve
policy_df["PolicyGap"] = policy_df["FedRate"] - policy_df["InflationGap"]

# create one melted dataframe for the inflation and fed
melted_government = policy_df[["observation_date","InflationRate", 'FedRate']].copy()

melted_government = melted_government.melt(
    id_vars="observation_date",
    value_vars=["InflationRate", "FedRate"],
    var_name="Metric",
    value_name="Value"
)

# create another melt chart just for the target gap and the inflation gap
gap_df = policy_df[["observation_date", "InflationGap", "PolicyGap"]].copy()

gap_df = gap_df.melt(
    id_vars="observation_date",
    value_vars=["InflationGap", "PolicyGap"],
    var_name="GapMetric",
    value_name="Value"
)

gov_color_scale = alt.Scale(
    domain=["InflationRate", "FedRate"],
    range=["#CB0000", "#007A7A"]
)

gap_color_scale = alt.Scale(
    domain=["InflationGap", "PolicyGap"],
    range=["#00A300", "#CB5C00"]
)

# for chart on the left
melt_gov = alt.Chart(melted_government).mark_line().encode(
    x=alt.X("observation_date:T", title=None),
    y=alt.Y("Value:Q", title="Value (%)"),
    color=alt.Color("Metric:N", scale=gov_color_scale,
        legend=alt.Legend(title="Metric"))
).properties(
    width=400,
    height=300,
    title="Inflation and the Federal Funds Rate"
).add_params(
    nearest
)

# Add dashed black line at y = 2% on the left chart
solid_target_line = alt.Chart(pd.DataFrame({"y": [2]})).mark_rule(
    color="black",
    strokeDash=[4, 4]
).encode(
    y="y:Q"
)

# Add dashed gray line at y=0 (2% inflation target on InflationGap scale)
target_line = alt.Chart(pd.DataFrame({"y": [0]})).mark_rule(
    color="gray",
    strokeDash=[4, 4]
).encode(
    y="y:Q"
)

# Label the target line (right chart)
target_label = alt.Chart(pd.DataFrame({
    "x": [policy_df["observation_date"].max()],
    "y": [0],
    "text": ["Target (2% inflation)"]
})).mark_text(
    align="left",
    baseline="bottom",
    dx=5,
    dy=-5,
    color="black"
).encode(
    x="x:T",
    y="y:Q",
    text="text:N"
)

# for chart on the right
gap_lines = alt.Chart(gap_df).mark_line().encode(
    x=alt.X("observation_date:T", title=None),
    y=alt.Y("Value:Q", title="Gap from 2% Inflation Target"),
    color=alt.Color("GapMetric:N", scale=gap_color_scale,
        legend=alt.Legend(title="Gap Type"))
).properties(
    width=400,
    height=300,
    title=""
).add_params(
    nearest
)

# combine gap lines with the target line and label
gap_chart = alt.layer(gap_lines, target_line, target_label)

# Label the target line (left chart)
true_label = alt.Chart(pd.DataFrame({
    "x": [policy_df["observation_date"].max()],
    "y": [2],
    "text": ["Target (2% inflation)"]
})).mark_text(
    align="left",
    baseline="bottom",
    dx=5,
    dy=-5,
    color="black"
).encode(
    x="x:T",
    y="y:Q",
    text="text:N"
)

# layer both the lines and label on the left chart
melt_got_line = alt.layer(melt_gov, solid_target_line, true_label)

# combine charts with fully independent color scales
policy_and_inflation_target = (melt_got_line & gap_chart).resolve_scale(color='independent')
policy_and_inflation_target


alt.VConcatChart(...)

In [ ]:
html = policy_and_inflation_target.to_html()

with open('/content/policy_and_inflation_target.html', 'w') as f:
    f.write(html)

In [ ]:
from google.colab import files
files.download('/content/policy_and_inflation_target.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
import altair as alt
import pandas as pd

# Step 1: Create target gap fields
policy_df = fedAinflation.copy()
policy_df["InflationGap"] = policy_df["InflationRate"] - 2
policy_df["PolicyGap"] = policy_df["FedRate"] - policy_df["InflationGap"]

# Step 2: Melt data for InflationRate and FedRate
melted_government = policy_df[["observation_date", "InflationRate", "FedRate"]].copy()
melted_government = melted_government.melt(
    id_vars="observation_date",
    value_vars=["InflationRate", "FedRate"],
    var_name="Metric",
    value_name="Value"
)

# Step 3: Create color scale
gov_color_scale = alt.Scale(
    domain=["InflationRate", "FedRate"],
    range=["#CB0000", "#007A7A"]
)

# Step 4: Create dynamic tooltip selection
nearest = alt.selection_point(
    nearest=True,
    on="pointerover",
    fields=["observation_date"],
    empty=False
)

# Step 5: Base line chart
line = alt.Chart(melted_government).mark_line().encode(
    x=alt.X("observation_date:T", title=None),
    y=alt.Y("Value:Q", title="Value (%)"),
    color=alt.Color("Metric:N", scale=gov_color_scale, legend=alt.Legend(title="Metric"))
)

# Step 6: Transparent selectors for hover
selectors = alt.Chart(melted_government).mark_point().encode(
    x="observation_date:T",
    opacity=alt.value(0)
).add_params(
    nearest
)

# Step 7: Points that show on hover
points = line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

# Step 8: Text labels that show values on hover
text = line.mark_text(align="left", dx=5, dy=-5).encode(
    text=alt.condition(nearest, "Value:Q", alt.value(" "))
)

# Step 9: Vertical rule that follows cursor
rules = alt.Chart(melted_government).mark_rule(color="gray").encode(
    x="observation_date:T"
).transform_filter(
    nearest
)

# Step 10: Dashed horizontal target line at 2%
solid_target_line = alt.Chart(pd.DataFrame({"y": [2]})).mark_rule(
    color="black",
    strokeDash=[4, 4]
).encode(
    y="y:Q"
)

# Step 11: Text label for the target line
true_label = alt.Chart(pd.DataFrame({
    "x": [policy_df["observation_date"].max()],
    "y": [2],
    "text": ["Target (2% inflation)"]
})).mark_text(
    align="left",
    baseline="bottom",
    dx=5,
    dy=-5,
    color="black"
).encode(
    x="x:T",
    y="y:Q",
    text="text:N"
)

# Step 12: Combine layers into interactive chart
melt_gov_line = alt.layer(
    line, selectors, points, rules, text
).properties(
    width=400,
    height=300,
    title="Inflation and the Federal Funds Rate"
)

# Step 13: Final layered chart with target line and label
melt_got_line = alt.layer(melt_gov_line, solid_target_line, true_label)

# This variable is now your updated interactive chart
melt_got_line


alt.LayerChart(...)

In [22]:
import altair as alt
import pandas as pd

# Step 1: Create gap fields
policy_df = fedAinflation.copy()
policy_df["InflationGap"] = policy_df["InflationRate"] - 2
policy_df["PolicyGap"] = policy_df["FedRate"] - policy_df["InflationGap"]

# Step 2: Melt datasets for plotting
melted_government = policy_df[["observation_date", "InflationRate", "FedRate"]].melt(
    id_vars="observation_date", var_name="Metric", value_name="Value"
)

gap_df = policy_df[["observation_date", "InflationGap", "PolicyGap"]].melt(
    id_vars="observation_date", var_name="GapMetric", value_name="Value"
)

# Color scales
gov_color_scale = alt.Scale(domain=["InflationRate", "FedRate"], range=["#CB0000", "#007A7A"])
gap_color_scale = alt.Scale(domain=["InflationGap", "PolicyGap"], range=["#00A300", "#CB5C00"])

# Shared tooltip selector
nearest = alt.selection_point(
    nearest=True,
    on="pointerover",
    fields=["observation_date"],
    empty=False
)


gov_line = alt.Chart(melted_government).mark_line().encode(
    x=alt.X("observation_date:T", title=None),
    y=alt.Y("Value:Q", title="Value (%)"),
    color=alt.Color("Metric:N", scale=gov_color_scale, legend=alt.Legend(title="Metric"))
)

gov_points = gov_line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

gov_text = gov_line.mark_text(align="left", dx=5, dy=-5).encode(
    text=alt.condition(nearest, "Value:Q", alt.value(" "))
)

gov_rule = alt.Chart(melted_government).mark_rule(color="gray").encode(
    x="observation_date:T"
).transform_filter(nearest)

gov_selectors = alt.Chart(melted_government).mark_point().encode(
    x="observation_date:T",
    opacity=alt.value(0)
).add_params(nearest)

gov_target_line = alt.Chart(pd.DataFrame({"y": [2]})).mark_rule(
    color="black", strokeDash=[4, 4]
).encode(y="y:Q")

gov_label = alt.Chart(pd.DataFrame({
    "x": [policy_df["observation_date"].max()],
    "y": [2],
    "text": ["Target (2% inflation)"]
})).mark_text(
    align="left", baseline="bottom", dx=5, dy=-5, color="black"
).encode(x="x:T", y="y:Q", text="text:N")

left_chart = alt.layer(
    gov_line, gov_selectors, gov_points, gov_rule, gov_text, gov_target_line, gov_label
).properties(
    width=400,
    height=300,
    title="Inflation and the Federal Funds Rate"
)


gap_line = alt.Chart(gap_df).mark_line().encode(
    x=alt.X("observation_date:T", title=None),
    y=alt.Y("Value:Q", title="Gap from 2% Inflation Target"),
    color=alt.Color("GapMetric:N", scale=gap_color_scale, legend=alt.Legend(title="Gap Type"))
)

gap_points = gap_line.mark_point().encode(
    opacity=alt.condition(nearest, alt.value(1), alt.value(0))
)

gap_text = gap_line.mark_text(align="left", dx=5, dy=-5).encode(
    text=alt.condition(nearest, "Value:Q", alt.value(" "))
)

gap_rule = alt.Chart(gap_df).mark_rule(color="gray").encode(
    x="observation_date:T"
).transform_filter(nearest)

gap_selectors = alt.Chart(gap_df).mark_point().encode(
    x="observation_date:T",
    opacity=alt.value(0)
).add_params(nearest)

gap_target_line = alt.Chart(pd.DataFrame({"y": [0]})).mark_rule(
    color="gray", strokeDash=[4, 4]
).encode(y="y:Q")

gap_label = alt.Chart(pd.DataFrame({
    "x": [policy_df["observation_date"].max()],
    "y": [0],
    "text": ["Target (2% inflation)"]
})).mark_text(
    align="left", baseline="bottom", dx=5, dy=-5, color="black"
).encode(x="x:T", y="y:Q", text="text:N")

right_chart = alt.layer(
    gap_line, gap_selectors, gap_points, gap_rule, gap_text, gap_target_line, gap_label
).properties(
    width=400,
    height=300,
    title=""
)

policy_and_inflation_target = (left_chart & right_chart).resolve_scale(color='independent')
policy_and_inflation_target


alt.VConcatChart(...)

In [ ]:
html = policy_and_inflation_target.to_html()

with open('/content/policy_and_inflation_target.html', 'w') as f:
    f.write(html)

In [ ]:
from google.colab import files
files.download('/content/policy_and_inflation_target.html')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>